# V2log 무게 플레이트 자동 라벨링

**Grounding DINO**로 원판 위치를 자동 감지하고, 폴더 이름으로 무게 클래스를 매핑합니다.

## 사용법
1. **런타임 → 런타임 유형 변경 → T4 GPU** 선택
2. 셀을 위에서 아래로 순서대로 실행 (▶ 버튼 또는 Shift+Enter)
3. Google Drive에 이미지 폴더를 먼저 업로드해두세요

### 예상 시간
- 설치: 3~5분
- 1,193장 처리: 30~60분
- Roboflow 업로드: 10~20분

---
## Cell 1: 패키지 설치
처음 한 번만 실행하면 됩니다. 3~5분 걸려요.

In [ ]:
!pip install -q transformers torch torchvision pillow tqdm roboflow supervision
print("설치 완료!")

---
## Cell 2: Google Drive 연결 + 설정
Google Drive에 접근 허용 팝업이 뜨면 **허용**을 눌러주세요.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ============================================
# 여기만 수정하세요!
# Google Drive에 업로드한 이미지 폴더 경로
# 예: My Drive > V2log-CV-Training > data > images > train
# ============================================
IMAGE_ROOT = "/content/drive/MyDrive/V2log-CV-Training/data/images/train"

# Roboflow 설정
ROBOFLOW_API_KEY = "5XX326NDaVcMsUiVJ726"
ROBOFLOW_WORKSPACE = "laptimev2log"
ROBOFLOW_PROJECT = "v2log-weight-plates"

# 결과 저장 경로 (Colab 내부, 수정 불필요)
OUTPUT_DIR = "/content/labeled_output"

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/images", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/labels", exist_ok=True)

# 폴더 확인
if os.path.exists(IMAGE_ROOT):
    folders = sorted(os.listdir(IMAGE_ROOT))
    total_images = 0
    for f in folders:
        fp = os.path.join(IMAGE_ROOT, f)
        if os.path.isdir(fp):
            count = len([x for x in os.listdir(fp) if x.lower().endswith(('.jpg','.jpeg','.png'))])
            total_images += count
            print(f"  {f}: {count}장")
    print(f"\n총 {total_images}장 발견!")
else:
    print(f"경로를 찾을 수 없습니다: {IMAGE_ROOT}")
    print("Google Drive에 이미지를 먼저 업로드하고, 위의 IMAGE_ROOT 경로를 수정하세요.")

---
## Cell 3: 클래스 매핑 설정
폴더 이름 → 무게 클래스를 연결합니다.

In [ ]:
# 클래스 정의 (순서 = 클래스 인덱스)
CLASS_NAMES = [
    "plate_2.5kg",  # 0
    "plate_5kg",    # 1
    "plate_10kg",   # 2
    "plate_15kg",   # 3
    "plate_20kg",   # 4
]

# 폴더 이름 → 클래스 인덱스 매핑
# 단일 플레이트 폴더: 감지된 모든 원판에 해당 클래스 부여
SINGLE_PLATE_FOLDERS = {
    "2.5kg":       0,
    "2.5kg 여러장": 0,
    "5kg":         1,
    "5kg 여러장":   1,
    "10gk":        2,
    "10kg 여러장":  2,
    "15kg":        3,
    "15kg 여러장":  3,
    "20kg":        4,
    "20gk 여러장":  4,
}

# 멀티 플레이트 폴더: 바운딩 박스만 감지, 클래스는 나중에 수동 지정
# (일단 class 99로 표시해서 Roboflow에서 수정)
MULTI_PLATE_FOLDERS = [
    "20kg 10kg",
    "20kg 10kg 5kg",
    "20kg 10kg 5 kg 2.5kg",
    "20kg 15kg",
    "20kg 15kg 10kg",
    "20kg 15kg 10kg 5kg",
    "20kg 15kg 10kg 5kg 2.5kg",
]

# 건너뛸 폴더
SKIP_FOLDERS = ["노이즈"]

print(f"단일 플레이트 폴더: {len(SINGLE_PLATE_FOLDERS)}개")
print(f"멀티 플레이트 폴더: {len(MULTI_PLATE_FOLDERS)}개 (나중에 수동 클래스 지정)")
print(f"건너뛸 폴더: {SKIP_FOLDERS}")

---
## Cell 4: Grounding DINO 모델 로드
AI 모델을 다운로드합니다. 처음 실행 시 1~2분 걸려요.

In [ ]:
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
import torch

MODEL_ID = "IDEA-Research/grounding-dino-tiny"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"GPU 사용: {DEVICE == 'cuda'} ({'T4' if DEVICE == 'cuda' else 'CPU - 느림!'}")
print(f"모델 다운로드 중: {MODEL_ID}...")

processor = AutoProcessor.from_pretrained(MODEL_ID)
model = AutoModelForZeroShotObjectDetection.from_pretrained(MODEL_ID).to(DEVICE)

print("모델 로드 완료!")

---
## Cell 5: 테스트 — 사진 1장으로 확인
본격적으로 돌리기 전에, 사진 1장에서 원판을 잘 찾는지 확인합니다.

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

def detect_plates(image_path, text_prompt="a round weight.",
                  box_threshold=0.25, text_threshold=0.25):
    """이미지에서 원판을 찾아서 바운딩 박스 좌표 반환"""
    image = Image.open(image_path).convert("RGB")
    w, h = image.size

    inputs = processor(images=image, text=text_prompt, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        outputs = model(**inputs)

    results = processor.post_process_grounded_object_detection(
        outputs,
        inputs.input_ids,
        box_threshold=box_threshold,
        text_threshold=text_threshold,
        target_sizes=[(h, w)]
    )[0]

    boxes = results["boxes"].cpu().numpy()
    scores = results["scores"].cpu().numpy()

    # YOLO 형식으로 변환 (정규화: 0~1)
    yolo_boxes = []
    for box, score in zip(boxes, scores):
        x1, y1, x2, y2 = box
        x_center = ((x1 + x2) / 2) / w
        y_center = ((y1 + y2) / 2) / h
        bw = (x2 - x1) / w
        bh = (y2 - y1) / h
        yolo_boxes.append({
            "x_center": x_center,
            "y_center": y_center,
            "width": bw,
            "height": bh,
            "score": float(score),
            "box_pixel": (float(x1), float(y1), float(x2), float(y2)),
        })

    return yolo_boxes


def visualize(image_path, boxes, class_name="plate"):
    """감지 결과를 시각화"""
    img = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(img)

    colors = ["#FF0000", "#00FF00", "#0000FF", "#FFFF00", "#FF00FF"]
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box["box_pixel"]
        color = colors[i % len(colors)]
        draw.rectangle([x1, y1, x2, y2], outline=color, width=3)
        draw.text((x1, y1 - 15), f"{class_name} ({box['score']:.2f})", fill=color)

    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"감지된 원판: {len(boxes)}개")
    plt.show()


# === 테스트: 첫 번째 폴더의 첫 번째 이미지 ===
test_folder = None
test_image = None

for fname in sorted(os.listdir(IMAGE_ROOT)):
    fpath = os.path.join(IMAGE_ROOT, fname)
    if os.path.isdir(fpath) and fname not in SKIP_FOLDERS:
        imgs = [x for x in sorted(os.listdir(fpath)) if x.lower().endswith(('.jpg','.jpeg','.png'))]
        if imgs:
            test_folder = fname
            test_image = os.path.join(fpath, imgs[0])
            break

if test_image:
    print(f"테스트 이미지: {test_folder}/{os.path.basename(test_image)}")
    boxes = detect_plates(test_image)
    print(f"감지된 원판: {len(boxes)}개")
    for i, b in enumerate(boxes):
        print(f"  #{i+1}: 위치=({b['x_center']:.2f}, {b['y_center']:.2f}), "
              f"크기=({b['width']:.2f} x {b['height']:.2f}), 확신도={b['score']:.2f}")
    visualize(test_image, boxes, test_folder)
else:
    print("테스트할 이미지를 찾을 수 없습니다. IMAGE_ROOT 경로를 확인하세요.")

---
## Cell 6: threshold 조정 (필요한 경우)

위 테스트 결과를 보고:
- 원판을 **못 찾았으면** → threshold를 **낮추세요** (0.15)
- **너무 많이** 찾았으면 (원판 아닌 것까지) → threshold를 **높이세요** (0.35)
- **잘 찾았으면** → 그대로 두고 다음 셀로!

In [ ]:
# ============================================
# threshold 조정 (기본값: 0.25)
# 원판을 못 찾으면 → 숫자를 낮추세요 (0.15)
# 원판이 아닌것까지 찾으면 → 숫자를 높이세요 (0.35)
# ============================================
BOX_THRESHOLD = 0.25
TEXT_THRESHOLD = 0.25

# 텍스트 프롬프트 (실제 헬스장 테스트에서 검증된 프롬프트)
TEXT_PROMPT = "a round weight."

print(f"설정: BOX_THRESHOLD={BOX_THRESHOLD}, TEXT_THRESHOLD={TEXT_THRESHOLD}")
print(f"프롬프트: '{TEXT_PROMPT}'")
print("\n다음 셀에서 전체 이미지 처리를 시작합니다.")

---
## Cell 7: 전체 이미지 자동 라벨링 (30~60분)
**이 셀이 핵심입니다!** 모든 이미지에서 원판을 찾고, YOLO 형식 라벨을 생성합니다.

In [ ]:
import shutil
from tqdm import tqdm
import time

# 결과 저장 디렉토리 초기화
for sub in ["images", "labels"]:
    d = os.path.join(OUTPUT_DIR, sub)
    if os.path.exists(d):
        shutil.rmtree(d)
    os.makedirs(d)

# 통계
stats = {
    "total": 0,
    "detected": 0,
    "no_detection": 0,
    "errors": 0,
    "per_folder": {},
}
no_detect_list = []

start_time = time.time()

# === 단일 플레이트 폴더 처리 ===
print("=" * 50)
print("단일 플레이트 폴더 처리 시작")
print("=" * 50)

for folder_name, class_idx in SINGLE_PLATE_FOLDERS.items():
    folder_path = os.path.join(IMAGE_ROOT, folder_name)
    if not os.path.isdir(folder_path):
        print(f"  [건너뜀] {folder_name} (폴더 없음)")
        continue

    images = sorted([f for f in os.listdir(folder_path)
                     if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

    if not images:
        continue

    class_name = CLASS_NAMES[class_idx]
    print(f"\n--- {folder_name} ({len(images)}장) → {class_name} ---")

    folder_detected = 0
    folder_no_detect = 0

    for img_file in tqdm(images, desc=folder_name):
        stats["total"] += 1
        img_path = os.path.join(folder_path, img_file)

        try:
            boxes = detect_plates(img_path, TEXT_PROMPT, BOX_THRESHOLD, TEXT_THRESHOLD)

            # 못 찾았으면 threshold 낮춰서 재시도
            if not boxes:
                boxes = detect_plates(img_path, TEXT_PROMPT, 0.15, 0.15)

            # 그래도 못 찾았으면 다른 프롬프트 시도
            if not boxes:
                for alt_prompt in ["round metal plate.", "weight plate.", "circular disc."]:
                    boxes = detect_plates(img_path, alt_prompt, 0.15, 0.15)
                    if boxes:
                        break

            # 파일 이름 (중복 방지: 폴더명_파일명)
            safe_folder = folder_name.replace(" ", "_")
            out_name = f"{safe_folder}__{img_file}"
            out_img = os.path.join(OUTPUT_DIR, "images", out_name)
            out_lbl = os.path.join(OUTPUT_DIR, "labels",
                                   os.path.splitext(out_name)[0] + ".txt")

            # 이미지 복사
            shutil.copy2(img_path, out_img)

            # YOLO 라벨 저장
            with open(out_lbl, "w") as f:
                if boxes:
                    for box in boxes:
                        f.write(f"{class_idx} {box['x_center']:.6f} {box['y_center']:.6f} "
                                f"{box['width']:.6f} {box['height']:.6f}\n")
                    stats["detected"] += 1
                    folder_detected += 1
                else:
                    # 감지 실패: 이미지 중앙에 기본 박스 (나중에 수동 수정)
                    f.write(f"{class_idx} 0.5 0.5 0.5 0.5\n")
                    stats["no_detection"] += 1
                    folder_no_detect += 1
                    no_detect_list.append(f"{folder_name}/{img_file}")

        except Exception as e:
            stats["errors"] += 1
            if stats["errors"] <= 5:
                print(f"  [에러] {img_file}: {e}")

    stats["per_folder"][folder_name] = {
        "total": len(images),
        "detected": folder_detected,
        "no_detect": folder_no_detect,
    }
    print(f"  → 감지: {folder_detected}/{len(images)} ({folder_no_detect}장 미감지)")

elapsed = time.time() - start_time
print(f"\n{'=' * 50}")
print(f"완료! (소요시간: {elapsed/60:.1f}분)")
print(f"총: {stats['total']} | 감지: {stats['detected']} | 미감지: {stats['no_detection']} | 에러: {stats['errors']}")

if no_detect_list:
    print(f"\n미감지 이미지 ({len(no_detect_list)}장):")
    for nd in no_detect_list[:20]:
        print(f"  - {nd}")
    if len(no_detect_list) > 20:
        print(f"  ... 외 {len(no_detect_list) - 20}장")

---
## Cell 8: 결과 시각화 (샘플 10장)
자동 라벨링 결과가 정확한지 확인합니다.

In [ ]:
import random

# 샘플 이미지 선택
all_labeled = [f for f in os.listdir(f"{OUTPUT_DIR}/images")
               if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
samples = random.sample(all_labeled, min(10, len(all_labeled)))

fig, axes = plt.subplots(2, 5, figsize=(25, 10))
axes = axes.flatten()

for i, img_name in enumerate(samples):
    img_path = os.path.join(OUTPUT_DIR, "images", img_name)
    lbl_path = os.path.join(OUTPUT_DIR, "labels",
                            os.path.splitext(img_name)[0] + ".txt")

    img = Image.open(img_path).convert("RGB")
    w, h = img.size
    draw = ImageDraw.Draw(img)

    # 라벨 파일 읽기
    if os.path.exists(lbl_path):
        with open(lbl_path) as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    cls_idx = int(parts[0])
                    xc, yc, bw, bh = float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])
                    # YOLO → pixel
                    x1 = (xc - bw/2) * w
                    y1 = (yc - bh/2) * h
                    x2 = (xc + bw/2) * w
                    y2 = (yc + bh/2) * h

                    colors = ["red", "green", "blue", "yellow", "magenta"]
                    color = colors[cls_idx % len(colors)]
                    draw.rectangle([x1, y1, x2, y2], outline=color, width=3)
                    cls_name = CLASS_NAMES[cls_idx] if cls_idx < len(CLASS_NAMES) else f"class_{cls_idx}"
                    draw.text((x1, y1-12), cls_name, fill=color)

    axes[i].imshow(img)
    axes[i].set_title(img_name[:30], fontsize=8)
    axes[i].axis("off")

plt.tight_layout()
plt.suptitle("자동 라벨링 결과 샘플", fontsize=16, y=1.02)
plt.show()

print("\n위 결과를 확인하세요:")
print("  - 원판 위에 박스가 정확하게 있나요?")
print("  - 원판이 아닌 곳에 박스가 있나요?")
print("  - 괜찮으면 다음 셀(Roboflow 업로드)을 실행하세요!")

---
## Cell 9: 폴더별 통계 확인

In [ ]:
print("폴더별 감지 결과:")
print(f"{'폴더':<20} {'총 이미지':>10} {'감지':>8} {'미감지':>8} {'감지율':>8}")
print("-" * 60)

for folder, data in stats["per_folder"].items():
    rate = data["detected"] / data["total"] * 100 if data["total"] > 0 else 0
    mark = "✅" if rate >= 80 else "⚠️" if rate >= 50 else "❌"
    print(f"{mark} {folder:<18} {data['total']:>10} {data['detected']:>8} {data['no_detect']:>8} {rate:>7.1f}%")

---
## Cell 10: Roboflow에 업로드
자동 라벨링된 이미지 + 바운딩 박스를 Roboflow에 업로드합니다.

**주의**: 기존 프로젝트에 이미지가 있으면 먼저 삭제하고 실행하세요.

In [ ]:
from roboflow import Roboflow

print("Roboflow 연결 중...")
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
print(f"프로젝트: {project.name}")

# 이미지 + 라벨 파일 목록
img_dir = os.path.join(OUTPUT_DIR, "images")
lbl_dir = os.path.join(OUTPUT_DIR, "labels")

images = sorted([f for f in os.listdir(img_dir)
                 if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

print(f"업로드할 이미지: {len(images)}장")

uploaded = 0
errors = 0

for img_file in tqdm(images, desc="Roboflow 업로드"):
    img_path = os.path.join(img_dir, img_file)
    lbl_file = os.path.splitext(img_file)[0] + ".txt"
    lbl_path = os.path.join(lbl_dir, lbl_file)

    try:
        if os.path.exists(lbl_path):
            project.upload(
                image_path=img_path,
                annotation_path=lbl_path,
                split="train",
            )
        else:
            project.upload(
                image_path=img_path,
                split="train",
            )
        uploaded += 1

        if uploaded % 50 == 0:
            print(f"  [{uploaded}장 완료]")

    except Exception as e:
        errors += 1
        if errors <= 5:
            print(f"  [에러] {img_file}: {e}")

print(f"\n{'=' * 40}")
print(f"업로드 완료! 성공: {uploaded} / 에러: {errors}")
print(f"\nRoboflow에서 확인하세요: https://app.roboflow.com/{ROBOFLOW_WORKSPACE}/{ROBOFLOW_PROJECT}")

---
## 다음 단계

1. Roboflow에서 **Annotate** 탭 확인
2. 박스가 잘못된 이미지만 수정 (삭제 후 다시 그리기)
3. 전체 리뷰 끝나면 **Generate Version** → train/valid/test 분할
4. YOLO26-N 모델 학습 시작!